In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
torch.cuda.empty_cache()
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix
# 设置是否使用GPU加速
from sklearn.decomposition import PCA
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.model_selection import TimeSeriesSplit
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import random
import pickle
import os
np.random.seed(42)
random.seed(42)

In [ ]:

def load_data(file_paths):
    dataframes = []
    for file_path in file_paths:
        df = pd.read_csv(file_path)
        dataframes.append(df)
    combined_df = pd.concat(dataframes, ignore_index=True)
    return combined_df


file_paths = [
    'D://数据//溢流数据交付//中值滤波//中值滤波结果18.csv',
    'D://数据//溢流数据交付//中值滤波//中值滤波结果17.csv',
    'D://数据//溢流数据交付//中值滤波//中值滤波结果25.csv',
    'D://数据//溢流数据交付//中值滤波//中值滤波结果10.csv',
    'D://数据//溢流数据交付//中值滤波//中值滤波结果16.csv',
    'D://数据//溢流数据交付//中值滤波//中值滤波结果6.csv',
    
]

combined_df = load_data(file_paths)


test_df_26 = pd.read_csv('D://数据//溢流数据交付//中值滤波//中值滤波结果27.csv')


def process_data(df):
    feature_columns =  ['出口密度', '总池体积', '入口密度', '立管压力', '出口流量(百分)', '大钩负荷', '钻压', '出口温度', '扭矩', '转盘转速']
    x = df[feature_columns].values
    y = df['溢流'].values
    return x, y


x_train, y_train = process_data(combined_df)


x_test, y_test = process_data(test_df_26)


def augment_data_with_borderline_smote(x, y):
    smote = BorderlineSMOTE(sampling_strategy=1)
    x_resampled, y_resampled = smote.fit_resample(x, y)
    return x_resampled, y_resampled


x_train_combined, y_train_combined = augment_data_with_borderline_smote(x_train, y_train)


scaler = MinMaxScaler()
x_train_normalized = scaler.fit_transform(x_train_combined)
x_test_normalized = scaler.transform(x_test)


pca = PCA(n_components=6)
x_train_pca = pca.fit_transform(x_train_normalized)
x_test_pca = pca.transform(x_test_normalized)


def save_preprocessor(preprocessors, filename):
    with open(filename, 'wb') as f:
        pickle.dump(preprocessors, f)


preprocessors = {
    'scaler': scaler,
    'pca': pca
}
save_preprocessor(preprocessors, 'preprocessor.pkl')

def print_class_distribution(y, dataset_name):
    unique, counts = np.unique(y, return_counts=True)
    distribution = dict(zip(unique, counts))
    print(f"{dataset_name} 类别分布:")
    for label, count in distribution.items():
        print(f"类别 {label}: {count} 个样本")



In [ ]:

param_ranges = [
    (1, 5), 
    (16, 128),  
    (0.00001, 0.05),  
    (20, 500)  
]


pop_size =20  
max_gru_layers = 5
N_GENERATIONS = 20
mutation_rate = 0.05
crossover_rate = 0.6
sequence_length = 540
input_size = 10
step_size=1
dropout_rate=0.5




In [ ]:
class TransformerEncoder(nn.Module):
    def __init__(self, input_size, num_heads, num_layers):
        super(TransformerEncoder, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=input_size, nhead=num_heads, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)

    def forward(self, x):
        return self.transformer_encoder(x)

class CustomGRUModel(nn.Module):
    def __init__(self, input_size, gru_units, transformer_layers, num_heads, dropout_rate=0.5):
        super(CustomGRUModel, self).__init__()
        self.transformer = TransformerEncoder(input_size=input_size, num_heads=num_heads, num_layers=transformer_layers)
        self.grus = nn.ModuleList()
        self.dropout = nn.Dropout(dropout_rate)
        for i in range(len(gru_units)):
            input_dim = input_size if i == 0 else gru_units[i - 1]
            self.grus.append(nn.GRU(input_size=input_dim, hidden_size=gru_units[i], num_layers=1, batch_first=True))
        self.fc = nn.Linear(gru_units[-1], 1)

    def forward(self, x):
        out = self.transformer(x)
        for gru in self.grus:
            out, _ = gru(out)
            out = self.dropout(out)
        return torch.sigmoid(self.fc(out[:, -1, :]))

def create_sliding_window(data, labels, sequence_length):
    x_windows = []
    y_windows = []

    for i in range(len(data) - sequence_length):
        window = data[i:i + sequence_length]
        x_windows.append(window)

        if labels is not None:
            future_label = max(labels[i + sequence_length:i + sequence_length + 1])
            y_windows.append(future_label)

    
    if labels is not None:
        return np.array(x_windows), np.array(y_windows)
    else:
        return np.array(x_windows), None  


def log_metrics(log_file, message):
    with open(log_file, 'a') as f:
        f.write(message + '\n')

def aim_function(x_train_combined, y_train_combined, x_test, y_test, sequence_length, num, n_splits=2):
    gru_layers = int(num[0])
    gru_units = [int(num[i + 1]) for i in range(gru_layers)]
    learning_rate = float(num[1 + gru_layers])
    num_epochs = int(num[2 + gru_layers])

    x_train_windows, y_train_windows = create_sliding_window(x_train_combined, y_train_combined, sequence_length)
    x_test_windows, y_test_windows = create_sliding_window(x_test, y_test, sequence_length)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    tscv = TimeSeriesSplit(n_splits=n_splits)
    fold_results = []

    for fold, (train_idx, val_idx) in enumerate(tscv.split(x_train_windows)):
        print(f'Fold {fold + 1}')

        train_dataset = TensorDataset(torch.tensor(x_train_windows[train_idx], dtype=torch.float32),
                                      torch.tensor(y_train_windows[train_idx], dtype=torch.float32))
        val_dataset = TensorDataset(torch.tensor(x_train_windows[val_idx], dtype=torch.float32),
                                    torch.tensor(y_train_windows[val_idx], dtype=torch.float32))

        batch_size = 256
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        model = CustomGRUModel(input_size=x_train_windows.shape[2], gru_units=gru_units, transformer_layers=2, num_heads=2).to(device)
        optimizer = optim.Adamax(model.parameters(), lr=learning_rate)
        criterion = nn.BCELoss()

        best_val_loss = float('inf')
        patience = 5
        patience_counter = 0

        for epoch in range(num_epochs):
            model.train()
            running_train_loss = 0.0
            for batch_x, batch_y in train_loader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device).float().unsqueeze(1)
                optimizer.zero_grad()
                output = model(batch_x)
                loss = criterion(output, batch_y)
                loss.backward()
                optimizer.step()
                running_train_loss += loss.item() * batch_x.size(0)

            avg_train_loss = running_train_loss / len(train_loader.dataset)
            print(f'Epoch {epoch + 1}/{num_epochs}, Training Loss: {avg_train_loss:.4f}')

            with torch.no_grad():
                val_loss = evaluate(val_loader, model, 0.43 , device, criterion)[-1]
                print(f'Epoch {epoch + 1}/{num_epochs}, Validation Loss: {val_loss:.4f}')

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
            else:
                patience_counter += 1

            if patience_counter >= patience:
                print(f'Early stopping at epoch {epoch + 1}')
                break

        val_accuracy, val_f1, val_recall, _, _, _, _, val_loss = evaluate(val_loader, model, 0.43, device, criterion)
        print(f'Validation Results for fold {fold + 1}: Accuracy={val_accuracy:.4f}, F1-score={val_f1:.4f}, Recall={val_recall:.4f}, Loss={val_loss:.4f}')
        fold_results.append((val_accuracy, val_f1, val_recall))

    avg_val_accuracy = np.mean([result[0] for result in fold_results])
    avg_val_f1 = np.mean([result[1] for result in fold_results])
    avg_val_recall = np.mean([result[2] for result in fold_results])
 
    print(f'Average Validation Accuracy: {avg_val_accuracy:.4f}')
    print(f'Average Validation F1-score: {avg_val_f1:.4f}')
    print(f'Average Validation Recall: {avg_val_recall:.4f}')

    test_dataset = TensorDataset(torch.tensor(x_test_windows, dtype=torch.float32), torch.tensor(y_test_windows, dtype=torch.float32))
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    test_accuracy, test_f1, test_recall, tn_test, fp_test, fn_test, tp_test, test_loss = evaluate(
        test_loader, model, 0.61, device, criterion, save_probs=True, filename="predicted_probs.txt"
    )
    test_message = (f'Final Test Accuracy: {test_accuracy:.4f}, Test F1-score: {test_f1:.4f}, '
                    f'Test Recall: {test_recall:.4f}, Final Test Loss: {test_loss:.4f}')
    print(test_message)
    log_metrics("training_log.txt", test_message)
    matrix_message = f'Test Confusion Matrix: TN={tn_test}, FP={fp_test}, FN={fn_test}, TP={tp_test}'
    print(matrix_message)
    log_metrics("training_log.txt", matrix_message)

    return test_accuracy

def evaluate(test_loader, model, threshold, device, criterion, save_probs=False, filename="predicted_probs.txt"):
    model.eval()
    all_outputs = []
    all_labels = []
    running_loss = 0.0

    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            output = model(batch_x)
            all_outputs.append(output.cpu().numpy())
            all_labels.append(batch_y.cpu().numpy())

            batch_y = batch_y.float().unsqueeze(1)
            loss = criterion(output, batch_y)
            running_loss += loss.item() * batch_x.size(0)

    avg_loss = running_loss / len(test_loader.dataset)

    all_outputs = np.concatenate(all_outputs)
    all_labels = np.concatenate(all_labels)

    probs = torch.sigmoid(torch.tensor(all_outputs)).numpy()
    preds = (probs >= threshold).astype(int)

    if save_probs:
        if os.path.exists(filename):
            existing_probs = np.loadtxt(filename)
            if existing_probs.ndim == 1:
                existing_probs = existing_probs.reshape(-1, 1)
            if probs.ndim == 1:
                probs = probs.reshape(-1, 1)
            if existing_probs.shape[1] != probs.shape[1]:
                print("Warning: Shape mismatch between existing and new probabilities. Skipping appending.")
                all_probs = existing_probs  # 使用现有概率
            else:
                all_probs = np.vstack((existing_probs, probs))
        else:
            if probs.ndim == 1:
                probs = probs.reshape(-1, 1)
            all_probs = probs  # 设置初始 all_probs

        # 确保在 save_probs 为 True 时，all_probs 一定被初始化
        np.savetxt(filename, all_probs, fmt='%.6f')
        print(f"Predicted probabilities saved to {filename}")

    accuracy = accuracy_score(all_labels, preds)
    recall = recall_score(all_labels, preds)
    f1 = f1_score(all_labels, preds)
    tn, fp, fn, tp = confusion_matrix(all_labels, preds, labels=[0, 1]).ravel()

    return accuracy, f1, recall, tn, fp, fn, tp, avg_loss

In [ ]:

def generate_individual(param_ranges, max_gru_layers):
    gru_layers = random.randint(1, max_gru_layers)  
    gru_units = [random.randint(param_ranges[1][0], param_ranges[1][1]) for _ in range(gru_layers)] 
    learning_rate = random.uniform(param_ranges[2][0], param_ranges[2][1]) 
    num_epochs = random.randint(param_ranges[3][0], param_ranges[3][1])  
    individual = [gru_layers] + gru_units + [learning_rate, num_epochs]
    return individual


def init_population(pop_size, max_gru_layers, param_ranges):
    return [generate_individual(param_ranges, max_gru_layers) for _ in range(pop_size)]


def encode_params(num):
    binary_encoding = format(num[0], '03b')  
    for i in range(1, 1 + num[0]):  
        binary_encoding += format(num[i], '08b')
    binary_encoding += format(int(num[1 + num[0]] * 1000), '011b') 
    binary_encoding += format(num[-1], '09b')  
    return binary_encoding


def decode_params(binary_encoding):
    decoded = [int(binary_encoding[:3], 2)]  
    for i in range(decoded[0]):
        start = 3 + i * 8
        decoded.append(int(binary_encoding[start:start + 8], 2))  

    lr_value = int(binary_encoding[3 + decoded[0] * 8:3 + decoded[0] * 8 + 11], 2) / 100000.0
    decoded_learning_rate = 0.00001 + lr_value * (0.0001 - 0.00001)
    decoded.append(decoded_learning_rate) 
    decoded.append(int(binary_encoding[-9:], 2)) 
    return decoded


def select(population, fitness):
    fitness = np.where(fitness == 0, 1e-6, fitness)  
    fitness = np.where(np.isnan(fitness), 1e-6, fitness)  
    cumulative_prob = np.cumsum(fitness / np.sum(fitness))
    r1, r2 = np.random.rand(2)

    parent1, parent2 = None, None
    for i, cp in enumerate(cumulative_prob):
        if parent1 is None and r1 < cp:
            parent1 = population[i]
        if parent2 is None and r2 < cp and parent1 is not population[i]:
            parent2 = population[i]
        if parent1 is not None and parent2 is not None:
            break

    if parent1 is None or parent2 is None:
        available_indices = list(range(len(population)))
        if parent1 is None:
            parent1 = population[random.choice(available_indices)]
        if parent2 is None:
            available_indices.remove(population.index(parent1))
            parent2 = population[random.choice(available_indices)]

    return parent1, parent2


def uniform_crossover(parent1, parent2, crossover_rate):
    if parent1 is None or parent2 is None:
        raise ValueError("Parents cannot be None")

    child = ''
    for i in range(len(parent1)):
        if np.random.rand() < crossover_rate:
            child += parent2[i]
        else:
            child += parent1[i]

    return child


def mutate(dna, mutation_rate):
    mutated = ''
    for bit in dna:
        if np.random.rand() < mutation_rate:
            mutated += '1' if bit == '0' else '0'
        else:
            mutated += bit
    return mutated


In [ ]:

def save_model(model, file_path):
    torch.save(model.state_dict(), file_path)

best_model_params = None
population = [encode_params(dna) for dna in init_population(pop_size, max_gru_layers, param_ranges)]
best_solution = None
best_fitness = -np.inf

for generation in range(N_GENERATIONS):
    fitness = np.zeros(pop_size)
    for i, dna in enumerate(population):
        params = decode_params(dna)
        print(f"Evaluating individual {i} with params: {params}")
        weighted_score = aim_function(x_train_combined, y_train_combined, x_test, y_test, sequence_length, num=params)
        fitness[i] = weighted_score
        print(f"Fitness[{i}] = {fitness[i]}")
       

    best_idx = np.argmax(fitness)
    if fitness[best_idx] > best_fitness:
        best_fitness = fitness[best_idx]
        best_solution = decode_params(population[best_idx])
        best_model_params = best_solution  

    print(f'Generation {generation + 1}, Best Fitness: {best_fitness:.4f}, Best Params: {best_solution}')

    # 选择父代进行交叉和变异
    new_population = []
    for _ in range(pop_size // 2):
        parent1, parent2 = select(population, fitness)
        child1_dna = uniform_crossover(parent1, parent2, crossover_rate)
        child2_dna = uniform_crossover(parent1, parent2, crossover_rate)
        new_population.append(mutate(child1_dna, mutation_rate))
        new_population.append(mutate(child2_dna, mutation_rate))

    population = new_population

if best_model_params is not None:
    print(f'Training final model with best parameters: {best_model_params}')
    gru_layers = int(best_model_params[0])
    gru_units = [int(best_model_params[i + 1]) for i in range(gru_layers)]
    learning_rate = float(best_model_params[1 + gru_layers])
    num_epochs = int(best_model_params[2 + gru_layers])

    model = CustomGRUModel(input_size=x_train_combined.shape[1], gru_units=gru_units, transformer_layers=2, num_heads=2).to(device)
    optimizer = optim.Adamax(model.parameters(), lr=learning_rate)
    criterion = nn.BCELoss()

    x_train_windows, y_train_windows = create_sliding_window(x_train_combined, y_train_combined, sequence_length)
    
    train_dataset = TensorDataset(torch.tensor(x_train_windows, dtype=torch.float32),
                                  torch.tensor(y_train_windows, dtype=torch.float32))
    train_loader = DataLoader(train_dataset, batch_size=256, shuffle=False)


    for epoch in range(num_epochs):
        model.train()
        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device).float().unsqueeze(1)
            optimizer.zero_grad()
            output = model(batch_x)
            loss = criterion(output, batch_y)
            loss.backward()
            optimizer.step()

    
model_params = {
    'input_size': x_train_combined.shape[1],
    'gru_units': gru_units,  
    'transformer_layers': 2,  
    'num_heads': 2  
}

torch.save({
    'model_state_dict': model.state_dict(),
    'model_params': model_params
}, 'best_gru_model.pth')

# 输出最佳结果
print(f'Best Parameters: {best_solution}')
print(f'Best Fitness Score: {best_fitness:.4f}')

In [ ]:
# 加载预处理参数
def load_preprocessor(filename):
    with open(filename, 'rb') as f:
        preprocessors = pickle.load(f)
    return preprocessors

preprocessors = load_preprocessor('preprocessor.pkl')
scaler = preprocessors['scaler']
pca = preprocessors['pca']

def load_model(file_path, device):
    checkpoint = torch.load(file_path)
    model_params = checkpoint['model_params']
    model = CustomGRUModel(
        input_size=model_params['input_size'],
        gru_units=model_params['gru_units'],
        transformer_layers=model_params['transformer_layers'],
        num_heads=model_params['num_heads']
    )
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)
    return model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
best_model = load_model('best_gru_model.pth', device)
def preprocess_data(df, scaler, pca):
    feature_columns =  ['出口密度', '总池体积', '入口密度', '立管压力', '出口流量(百分)', '大钩负荷', '钻压', '出口温度', '扭矩', '转盘转速']
    x = df[feature_columns].values
    x_normalized = scaler.transform(x)
    x_pca = pca.transform(x_normalized)
    return x_pca

# 加载并处理新数据
data_path = "D://数据//溢流数据交付//中值滤波//中值滤波结果4.csv"
new_data = pd.read_csv(data_path)
processed_data = preprocess_data(new_data, scaler, pca)

true_labels = new_data['溢流'].values  

def create_sliding_window(data, labels, sequence_length, stride=1):
    x_windows = []
    y_windows = []
    for i in range(0, len(data) - sequence_length + 1, stride):
        x_windows.append(data[i:i + sequence_length])
        if labels is not None:
            y_windows.append(labels[i + sequence_length - 1])
    return np.array(x_windows), (np.array(y_windows) if labels is not None else None)

# 创建滑动窗口的数据
sequence_length = 540
x_windows, y_windows = create_sliding_window(processed_data, true_labels, sequence_length)

x_windows_tensor = torch.tensor(x_windows, dtype=torch.float32).to(device)

best_model.eval()

with torch.no_grad():
    predictions = best_model(x_windows_tensor)

probabilities = torch.sigmoid(predictions).cpu().numpy()

threshold = 0.3
predicted_classes = (probabilities >= threshold).astype(int).flatten()

# 打印预测概率
print("Predicted probabilities:\n", probabilities)

# 确保长度匹配
print(f"Length of y_windows: {len(y_windows)}")
print(f"Length of predicted_classes: {len(predicted_classes)}")

# 将预测概率保存到 CSV 文件
probabilities_df = pd.DataFrame(probabilities, columns=['Probability'])  # 可以调整列名
probabilities_df.to_csv('predicted_probabilities.csv', index=False)

# 计算准确率
accuracy = accuracy_score(y_windows, predicted_classes)

# 计算召回率
recall = recall_score(y_windows, predicted_classes)

# 计算F1分数
f1 = f1_score(y_windows, predicted_classes)

# 计算混淆矩阵
conf_matrix = confusion_matrix(y_windows, predicted_classes)

print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:\n", conf_matrix)